In [ ]:
import os
import requests
import zipfile
import cv2
import tensorflow as tf
import keras
from keras import layers
import matplotlib.pyplot as plt
import numpy

#hyper params
batch_size = 10

# defineing utility methods
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# unzipfile
with zipfile.ZipFile("/content/drive/My Drive/tif_extended.zip", 'r') as zip_ref:
    zip_ref.extractall("./trainingSrc")

PATH = "./trainingSrc/tif_extended"
CLASSES = os.listdir(PATH)

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split = 0.2)
images = image_generator.flow_from_directory(
    PATH,
    batch_size = 1,
    target_size = (256, 256),
)
sample, label = next(images)
feature_dataframe = []
target_dataframe = []

for i in range(len(images)):
  feature_dataframe.append(images[i][0][0])
  target_dataframe.append(images[i][1][0])

feature_dataframe = numpy.array(feature_dataframe)
target_dataframe = numpy.array(target_dataframe)

# create network
model = keras.Sequential()

# input and first convolution: extract 30 features
model.add(keras.layers.Conv2D(30, 2, activation="relu", input_shape = (256, 256, 3)))
model.add(keras.layers.Conv2D(60, kernel_size=5, strides=(2, 2), activation="relu"))#(60, 5, stri activation="relu"))
# input and second convolution: extract 30 features
model.add(keras.layers.Conv2D(60, 5, activation="relu"))
model.add(keras.layers.Conv2D(30, 3, activation="relu"))
model.add(keras.layers.MaxPooling2D(2))

# input and third convolution: extract 30 features
model.add(keras.layers.Conv2D(60, 5, activation="relu"))
model.add(keras.layers.MaxPooling2D(2))

#flatten
model.add(keras.layers.Flatten())
# three dense layers
model.add(keras.layers.Dense(120, activation="relu"))
model.add(keras.layers.Dense(28, activation="relu"))
model.add(keras.layers.Dense(7, activation="softmax"))

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["acc"]
)

model.summary()

In [ ]:
model.fit(
    x = feature_dataframe,
    y = target_dataframe,
    batch_size = batch_size,
    epochs = 10,
    validation_split = 0.2
)

In [ ]:
model.predict(feature_dataframe[50:55])

In [ ]:
plotImages(feature_dataframe[50:55])


In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
